In [1]:
from os import listdir
from os.path import isfile, join
from typing import List, Dict
import string
import random
import pdf_lines_gluer

In [2]:
def load_texts():
    f_names = [join('corpus', f) for f in listdir('corpus') if isfile(join('corpus', f))]
    f_names = [f for f in f_names if f.lower().endswith('.txt')]
    print(f_names)
    for fn in f_names:
        with open(fn, 'rt', encoding='utf-8', errors='replace') as f:
            text = f.read()
            if not text[0] in {'+', '*'}:
                print(f"File '{fn}' is not annotated, skipped.")
                continue
            print(f"File '{fn}' is annotated, add to collection.")
            yield text
            
raw_corpus = list(load_texts())
print(len(raw_corpus))

['corpus\\1005058.txt', 'corpus\\1005395.txt', 'corpus\\104888.txt', 'corpus\\105529.txt', 'corpus\\200850.txt', 'corpus\\200851.txt', 'corpus\\300125.txt', 'corpus\\300138.txt', 'corpus\\500150.txt', 'corpus\\500486.txt', 'corpus\\601777.txt', 'corpus\\601779.txt']
File 'corpus\1005058.txt' is annotated, add to collection.
File 'corpus\1005395.txt' is annotated, add to collection.
File 'corpus\104888.txt' is not annotated, skipped.
File 'corpus\105529.txt' is not annotated, skipped.
File 'corpus\200850.txt' is annotated, add to collection.
File 'corpus\200851.txt' is not annotated, skipped.
File 'corpus\300125.txt' is not annotated, skipped.
File 'corpus\300138.txt' is not annotated, skipped.
File 'corpus\500150.txt' is not annotated, skipped.
File 'corpus\500486.txt' is not annotated, skipped.
File 'corpus\601777.txt' is not annotated, skipped.
File 'corpus\601779.txt' is not annotated, skipped.
3


In [3]:
x, y = pdf_lines_gluer._featurize_text_with_annotation(raw_corpus[0])

In [4]:
print(x[:10])

[{'this_len': 12, 'mean_len': 75.0, 'prev_len': 0, 'prev_glued': 0, 'first_chars': 'Aa', 'isalpha': False, 'isdigit': False, 'islower': False, 'punct': ' '}, {'this_len': 97, 'mean_len': 79.33333333333333, 'prev_len': 0, 'prev_glued': 0, 'first_chars': 'Aa', 'isalpha': False, 'isdigit': False, 'islower': False, 'punct': ' '}, {'this_len': 104, 'mean_len': 82.71428571428571, 'prev_len': 11, 'prev_glued': 0, 'first_chars': 'aa', 'isalpha': False, 'isdigit': False, 'islower': False, 'punct': '.'}, {'this_len': 62, 'mean_len': 79.875, 'prev_len': 11, 'prev_glued': 1, 'first_chars': 'a-', 'isalpha': False, 'isdigit': False, 'islower': False, 'punct': '.'}, {'this_len': 100, 'mean_len': 81.88888888888889, 'prev_len': 11, 'prev_glued': 2, 'first_chars': 'Aa', 'isalpha': False, 'isdigit': False, 'islower': False, 'punct': '.'}, {'this_len': 101, 'mean_len': 84.2, 'prev_len': 11, 'prev_glued': 0, 'first_chars': 'aa', 'isalpha': False, 'isdigit': False, 'islower': False, 'punct': '.'}, {'this_le

In [6]:
xx, yy = [], []
for raw_text in raw_corpus:
    x, y = pdf_lines_gluer._featurize_text_with_annotation(raw_text)
    xx+=x
    yy+=y
print(f"Total samples: {len(yy)}")
print(f"Positive samples: {sum(y for y in yy if y)}")

Total samples: 2300
Positive samples: 1611


In [7]:
random.seed(1974)
combined = list(zip(xx, yy))
random.shuffle(combined)
xx[:], yy[:] = zip(*combined)

In [8]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split

In [9]:
v = DictVectorizer(sparse=False)
v.fit(xx)

DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
        sparse=False)

In [10]:
xx_features = v.transform(xx)
print(xx_features[:1])

[[ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.
  39.1  1.  30.   0.   1.  36. ]]


In [11]:
x_train, x_test, y_train, y_test = train_test_split(xx_features, yy, test_size=0.3, random_state=1974)

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [13]:
#clf = RandomForestClassifier(random_state=1974)
clf = LogisticRegression(random_state=1974, solver='liblinear', max_iter=2000,
                         # class_weight='balanced'
                        )
clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)

print(classification_report(y_true=y_test, y_pred=y_pred))

              precision    recall  f1-score   support

       False       0.79      0.86      0.82       207
        True       0.94      0.90      0.92       483

   micro avg       0.89      0.89      0.89       690
   macro avg       0.86      0.88      0.87       690
weighted avg       0.89      0.89      0.89       690



## Checking

In [15]:
text = """The rapid expansion of wireless services such as cellular voice, PCS
(Personal Communications Services), mobile data and wireless LANs
in recent years is an indication that signicant value is placed on accessibility
and portability as key features of telecommunication (Salkintzis and Mathiopoulos (Guest Ed.), 2000).
devices have maximum utility when they can be used any-
where at anytime". One of the greatest limitations to that goal, how-
ever, is nite power supplies. Since batteries provide limited power, a
general constraint of wireless communication is the short continuous
operation time of mobile terminals. Therefore, power management is
y Corresponding Author: Dr. Krishna Sivalingam. Part of the research was
supported by Air Force Oce of Scientic Research grants F-49620-97-1-
0471 and F-49620-99-1-0125; by Telcordia Technologies and by Intel. Part of
the work was done while the rst author was at Washington State Univer-
sity. The authors' can be reached at cej@bbn.com, krishna@eecs.wsu.edu,
pagrawal@research.telcordia.com, jcchen@research.telcordia.com
c
2001 Kluwer Academic Publishers. Printed in the Netherlands.
Jones, Sivalingam, Agrawal and Chen
one of the most challenging problems in wireless communication, and
recent research has addressed this topic (Bambos, 1998). Examples include
a collection of papers available in (Zorzi (Guest Ed.), 1998) and
a recent conference tutorial (Srivastava, 2000), both devoted to energy
ecient design of wireless networks.
Studies show that the signicant consumers of power in a typical
laptop are the microprocessor (CPU), liquid crystal display (LCD),
hard disk, system memory (DRAM), keyboard/mouse, CDROM drive,
oppy drive, I/O subsystem, and the wireless network interface card
(Udani and Smith, 1996, Stemm and Katz, 1997). A typical example
from a Toshiba 410 CDT mobile computer demonstrates that nearly
36% of power consumed is by the display, 21% by the CPU/memory,
18% by the wireless interface, and 18% by the hard drive. Consequently,
energy conservation has been largely considered in the hardware design
of the mobile terminal (Chandrakasan and Brodersen, 1995) and in
components such as CPU, disks, displays, etc. Signicant additional
power savings may result by incorporating low-power strategies into
the design of network protocols used for data communication. This
paper addresses the incorporation of energy conservation at all layers
of the protocol stack for wireless networks.
The remainder of this paper is organized as follows. Section 2 introduces
the network architectures and wireless protocol stack considered
in this paper. Low-power design within the physical layer is brie
y
discussed in Section 2.3. Sources of power consumption within mobile
terminals and general guidelines for reducing the power consumed are
presented in Section 3. Section 4 describes work dealing with energy
ecient protocols within the MAC layer of wireless networks, and
power conserving protocols within the LLC layer are addressed in Section
5. Section 6 discusses power aware protocols within the network
layer. Opportunities for saving battery power within the transport
layer are discussed in Section 7. Section 8 presents techniques at the
OS/middleware and application layers for energy ecient operation.
Finally, Section 9 summarizes and concludes the paper.
2. Background
This section describes the wireless network architectures considered in
this paper. Also, a discussion of the wireless protocol stack is included
along with a brief description of each individual protocol layer. The
physical layer is further discussed. """

In [17]:
corrected = pdf_lines_gluer._preprocess_pdf(text, clf, v)
print(corrected)

The rapid expansion of wireless services such as cellular voice, PCS (Personal Communications Services), mobile data and wireless LANs in recent years is an indication that signicant value is placed on accessibility and portability as key features of telecommunication (Salkintzis and Mathiopoulos (Guest Ed.), 2000). devices have maximum utility when they can be used anywhere at anytime". One of the greatest limitations to that goal, however, is nite power supplies. Since batteries provide limited power, a general constraint of wireless communication is the short continuous operation time of mobile terminals. Therefore, power management is y Corresponding Author: Dr. Krishna Sivalingam. Part of the research was supported by Air Force Oce of Scientic Research grants F-49620-97-10471 and F-49620-99-1-0125; by Telcordia Technologies and by Intel. Part of the work was done while the rst author was at Washington State University. The authors' can be reached at cej@bbn.com, krishna@eecs.wsu.e

In [18]:
ru_text = """Метод опорных векторов предназначен для решения задач клас-
сификации путем поиска хороших решающих границ (рис. 1.10), 
разделяющих два набора точек, принадлежащих разным катего-
риям. Решающей границей может быть линия или поверхность, 
разделяющая выборку обучающих данных на пространства, при-
надлежащие двум категориям. Для классификации новых точек 
достаточно только проверить, по какую сторону от границы они 
находятся.
Поиск таких границ метод опорных векторов осуществляет в два 
этапа:
1. Данные отображаются в новое пространство более высокой 
размерности, где граница может быть представлена как гипер-
плоскость (если данные были двумерными, как на рис. 1.10, 
гиперплоскость вырождается в линию).
2. Хорошая решающая граница (разделяющая гиперплоскость) вычисляется
путем максимизации расстояния от гиперплоскости до ближайших точек 
каждого класса, этот этап называют максимизацией зазора. Это позволяет 
обобщить классификацию новых образцов, не принадлежащих обучающему 
набору данных."""

In [19]:
corrected = pdf_lines_gluer._preprocess_pdf(ru_text, clf, v)
print(corrected)

Метод опорных векторов предназначен для решения задач классификации путем поиска хороших решающих границ (рис. 1.10), разделяющих два набора точек, принадлежащих разным категориям. Решающей границей может быть линия или поверхность, разделяющая выборку обучающих данных на пространства, принадлежащие двум категориям. Для классификации новых точек достаточно только проверить, по какую сторону от границы они находятся.
Поиск таких границ метод опорных векторов осуществляет в два этапа:
1. Данные отображаются в новое пространство более высокой размерности, где граница может быть представлена как гиперплоскость (если данные были двумерными, как на рис. 1.10, гиперплоскость вырождается в линию).
2. Хорошая решающая граница (разделяющая гиперплоскость) вычисляется путем максимизации расстояния от гиперплоскости до ближайших точек каждого класса, этот этап называют максимизацией зазора. Это позволяет обобщить классификацию новых образцов, не принадлежащих обучающему набору данных.


## Check minimal properties set to save vectorizer and classifier

In [21]:
print(repr(v))

DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
        sparse=False)


In [33]:
print(v.feature_names_)
print(v.vocabulary_)

['first_chars= ', 'first_chars="a', "first_chars=' ", "first_chars='A", 'first_chars=(0', 'first_chars=(A', 'first_chars=(a', 'first_chars=)]', 'first_chars=, ', 'first_chars=. ', 'first_chars=0', 'first_chars=0 ', 'first_chars=0,', 'first_chars=0.', 'first_chars=00', 'first_chars=0:', 'first_chars=0\\', 'first_chars=@', 'first_chars=A', 'first_chars=A ', 'first_chars=A,', 'first_chars=A-', 'first_chars=A.', 'first_chars=A0', 'first_chars=A=', 'first_chars=AA', 'first_chars=Aa', 'first_chars=[0', 'first_chars=[A', 'first_chars=[a', 'first_chars=\\A', 'first_chars=a ', 'first_chars=a(', 'first_chars=a-', 'first_chars=a.', 'first_chars=a0', 'first_chars=aA', 'first_chars=a[', 'first_chars=aa', 'isalpha', 'isdigit', 'islower', 'mean_len', 'prev_glued', 'prev_len', 'punct= ', 'punct=.', 'this_len']
{'first_chars= ': 0, 'first_chars="a': 1, "first_chars=' ": 2, "first_chars='A": 3, 'first_chars=(0': 4, 'first_chars=(A': 5, 'first_chars=(a': 6, 'first_chars=)]': 7, 'first_chars=, ': 8, 'firs

In [43]:
vv = DictVectorizer()
vv.feature_names_ = v.feature_names_
vv.vocabulary_ = v.vocabulary_

In [59]:
print(repr(clf.coef_))
print(repr(clf.classes_))
print(clf.intercept_)

array([[-1.92428293, -0.57402104,  0.2315162 ,  0.22349134, -0.21893115,
         0.40302487,  1.06171059, -0.3000657 , -0.80354834, -0.59474902,
         0.54936339, -0.95955689,  0.65555019, -1.90063708,  0.72334064,
        -0.17199479,  0.15464983,  0.31376216, -0.66997325, -1.18498478,
        -0.270456  ,  0.        , -1.5816989 , -0.43472727,  0.49956562,
        -0.46305976, -1.72074182,  0.27111279,  0.22436558,  0.        ,
         1.32364416,  0.04193252, -0.14462101,  1.64987425,  0.89867317,
         0.50371394,  0.2192005 , -0.50426949,  2.57259903,  0.        ,
         0.        ,  0.        ,  0.02742372,  0.07301828,  0.03715351,
        -0.49300996, -1.40821851,  0.01243852]])
array([False,  True])
[-1.90122847]


In [45]:
clf2 = LogisticRegression()
clf2.coef_ = clf.coef_
clf2.classes_ = clf.classes_
clf2.intercept_ = clf.intercept_

In [46]:
corrected = pdf_lines_gluer._preprocess_pdf(ru_text, clf2, vv)
print(corrected)

Метод опорных векторов предназначен для решения задач классификации путем поиска хороших решающих границ (рис. 1.10), разделяющих два набора точек, принадлежащих разным категориям. Решающей границей может быть линия или поверхность, разделяющая выборку обучающих данных на пространства, принадлежащие двум категориям. Для классификации новых точек достаточно только проверить, по какую сторону от границы они находятся.
Поиск таких границ метод опорных векторов осуществляет в два этапа:
1. Данные отображаются в новое пространство более высокой размерности, где граница может быть представлена как гиперплоскость (если данные были двумерными, как на рис. 1.10, гиперплоскость вырождается в линию).
2. Хорошая решающая граница (разделяющая гиперплоскость) вычисляется путем максимизации расстояния от гиперплоскости до ближайших точек каждого класса, этот этап называют максимизацией зазора. Это позволяет обобщить классификацию новых образцов, не принадлежащих обучающему набору данных.


In [49]:
print(clf.predict_proba([x_test[0]]))
print(clf2.predict_proba([x_test[0]]))

[[0.01549817 0.98450183]]
[[0.01549817 0.98450183]]


## Serialize as code

In [65]:
import datetime

serialized_as_code = f"""
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from numpy import array

_clf = LogisticRegression()
_clf.coef_ = {repr(clf.coef_)}
_clf.classes_ = {repr(clf.classes_)}
_clf.intercept_ = {clf.intercept_}

_v = DictVectorizer()
_v.feature_names_ = {v.feature_names_}
_v.vocabulary_ = {v.vocabulary_}

def preprocess_pdf(text: str) -> str:
    return _preprocess_pdf(text, _clf, _v)

"""

serialized_as_code = f"\n# This code was automatically generated at {datetime.datetime.now()}\n"+\
    serialized_as_code+\
    "# end of automatically generated code"

In [66]:
print(serialized_as_code)


# This code was automatically generated at 2019-05-17 17:56:24.195381

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from numpy import array

_clf = LogisticRegression()
_clf.coef_ = array([[-1.92428293, -0.57402104,  0.2315162 ,  0.22349134, -0.21893115,
         0.40302487,  1.06171059, -0.3000657 , -0.80354834, -0.59474902,
         0.54936339, -0.95955689,  0.65555019, -1.90063708,  0.72334064,
        -0.17199479,  0.15464983,  0.31376216, -0.66997325, -1.18498478,
        -0.270456  ,  0.        , -1.5816989 , -0.43472727,  0.49956562,
        -0.46305976, -1.72074182,  0.27111279,  0.22436558,  0.        ,
         1.32364416,  0.04193252, -0.14462101,  1.64987425,  0.89867317,
         0.50371394,  0.2192005 , -0.50426949,  2.57259903,  0.        ,
         0.        ,  0.        ,  0.02742372,  0.07301828,  0.03715351,
        -0.49300996, -1.40821851,  0.01243852]])
_clf.classes_ = array([False,  True])
_clf.interce

In [67]:
with open('pdf_lines_gluer.py', 'r', encoding='utf-8') as file:
    template = file.read()

generated_code = template.replace('# inject code here #', serialized_as_code)

with open('pdf_preprocessor.py', 'wt', encoding='utf-8') as file:
    file.write(generated_code)

In [68]:
print(generated_code)

import string
from typing import List, Dict

# This code was automatically generated at 2019-05-17 17:56:24.195381

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from numpy import array

_clf = LogisticRegression()
_clf.coef_ = array([[-1.92428293, -0.57402104,  0.2315162 ,  0.22349134, -0.21893115,
         0.40302487,  1.06171059, -0.3000657 , -0.80354834, -0.59474902,
         0.54936339, -0.95955689,  0.65555019, -1.90063708,  0.72334064,
        -0.17199479,  0.15464983,  0.31376216, -0.66997325, -1.18498478,
        -0.270456  ,  0.        , -1.5816989 , -0.43472727,  0.49956562,
        -0.46305976, -1.72074182,  0.27111279,  0.22436558,  0.        ,
         1.32364416,  0.04193252, -0.14462101,  1.64987425,  0.89867317,
         0.50371394,  0.2192005 , -0.50426949,  2.57259903,  0.        ,
         0.        ,  0.        ,  0.02742372,  0.07301828,  0.03715351,
        -0.49300996, -1.40821851,  0.01243852]])
_clf.c

In [64]:
print(type(clf.coef_))

<class 'numpy.ndarray'>


## Check generated file

In [70]:
from pdf_preprocessor import preprocess_pdf

In [71]:
print(preprocess_pdf(ru_text))

Метод опорных векторов предназначен для решения задач классификации путем поиска хороших решающих границ (рис. 1.10), разделяющих два набора точек, принадлежащих разным категориям. Решающей границей может быть линия или поверхность, разделяющая выборку обучающих данных на пространства, принадлежащие двум категориям. Для классификации новых точек достаточно только проверить, по какую сторону от границы они находятся.
Поиск таких границ метод опорных векторов осуществляет в два этапа:
1. Данные отображаются в новое пространство более высокой размерности, где граница может быть представлена как гиперплоскость (если данные были двумерными, как на рис. 1.10, гиперплоскость вырождается в линию).
2. Хорошая решающая граница (разделяющая гиперплоскость) вычисляется путем максимизации расстояния от гиперплоскости до ближайших точек каждого класса, этот этап называют максимизацией зазора. Это позволяет обобщить классификацию новых образцов, не принадлежащих обучающему набору данных.


In [72]:
print(preprocess_pdf(text))

The rapid expansion of wireless services such as cellular voice, PCS (Personal Communications Services), mobile data and wireless LANs in recent years is an indication that signicant value is placed on accessibility and portability as key features of telecommunication (Salkintzis and Mathiopoulos (Guest Ed.), 2000). devices have maximum utility when they can be used anywhere at anytime". One of the greatest limitations to that goal, however, is nite power supplies. Since batteries provide limited power, a general constraint of wireless communication is the short continuous operation time of mobile terminals. Therefore, power management is y Corresponding Author: Dr. Krishna Sivalingam. Part of the research was supported by Air Force Oce of Scientic Research grants F-49620-97-10471 and F-49620-99-1-0125; by Telcordia Technologies and by Intel. Part of the work was done while the rst author was at Washington State University. The authors' can be reached at cej@bbn.com, krishna@eecs.wsu.e